# HR Employee Attrition Analysis,
    This notebook performs exploratory data analysis (EDA) and insights extraction on the WA_Fn-UseC_-HR-Employee-Attrition dataset using both Pandas and SQL (SQLite)."
   

In [ ]:
import pandas as pd
import sqlite3
import plotly.express as px
# Load dataset
df = pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition.csv')
df.head()

## Setup SQLite database

In [ ]:
conn = sqlite3.connect('hr.db')
cursor = conn.cursor()
# Insert DataFrame into SQLite table named 'employee'
# Uncomment the next line if you need to insert the data first
# df.to_sql('employee', conn, index=False, if_exists='replace'

## 1. Employees who have not left the company

In [ ]:
print("1. How many employees are there?")
# SQL way
query='''
SELECT COUNT(*) FROM employee WHERE Attrition = 'No';'''
cursor.execute(query)
results = cursor.fetchall() 
print("SQL:", results[0][0])
# Pandas way
employees=df[df['Attrition'] == 'No'].shape[0]
print("Pandas:", employees)

## 2. Employee count per department

In [ ]:
print("2. What is the employee count for each department?")
query2 = '''SELECT Department, COUNT(*) as EmployeeCount
FROM employee
GROUP BY Department;'''
cursor.execute(query2)
results = cursor.fetchall() 
print("Employee count in eachtment:")
for department, count in results:
    print(f"{department}: {count}")
# Pandas way
dept_counts = df.groupby('Department').size()
print("Pandas:")
for department, count in dept_counts.items():
    print(f"{department}: {count}")

## 3. Average monthly income per job role

In [ ]:
print(" 3. What is the average monthly income for employees in each job role?")
query3 = ''' SELECT JobRole, AVG(MonthlyIncome) as AverageMonthlyIncome FROM employee GROUP BY JobRole; '''
cursor.execute(query3)
results = cursor.fetchall()  
print("Average monthly income for each job role:")
for job_role, avg_income in results:
    print(f"{job_role}: ${avg_income:.2f}")
# Pandas way
avg_income_roles = df.groupby('JobRole')['MonthlyIncome'].mean()
print("Pandas:")
for job_role, avg_income in avg_income_roles.items():
    print(f"{job_role}: ${avg_income:.2f}")

## 4. Top 5 employees by performance rating

In [ ]:
print("4. who are the top 5 employees by performance rating?")
#SQl
query4 = ''' SELECT EmployeeNumber, PerformanceRating FROM employee 
ORDER BY PerformanceRating DESC LIMIT 5; '''
cursor.execute(query4)
results = cursor.fetchall()  
print("the top 5 employees by performance rating:")
for emp_num, rating in results:
    print(f"EmployeeNumber: {emp_num}, PerformanceRating: {rating}")
# Pandas way
top5 = df.sort_values('PerformanceRating', ascending=False)[['EmployeeNumber', 'PerformanceRating']].head(5)
print("Pandas:")
for _, row in top5.iterrows():
    print(f"EmployeeNumber: {row['EmployeeNumber']}, PerformanceRating: {row['PerformanceRating']}")

## 5. Department with highest average performance rating

In [ ]:
print("5. which department has the highest average performance rating?")
#SQL
query5 = ''' SELECT Department, AVG(PerformanceRating) as AvgPerformanceRating
FROM employee GROUP BY Department ORDER BY AvgPerformanceRating DESC LIMIT 1; '''
cursor.execute(query5)
results = cursor.fetchall()
print("Department with the highest average performance rating:")
for department, avg_rating in results:
    print(f"{department}: {avg_rating:.2f}")
# Pandas way
dept_avg_perf = df.groupby('Department')['PerformanceRating'].mean()
best_dept = dept_avg_perf.idxmax()
best_rating = dept_avg_perf.max()
print("Pandas:")
print(f"{best_dept}: {best_rating:.2f}")

## 6. Attrition rate by job role and department

In [ ]:
print("6. Attrition rate by job role and department")
# SQL way
query6 = """
SELECT Department, JobRole,
       ROUND(SUM(CASE WHEN Attrition='Yes' THEN 1 ELSE 0 END)*100.0/COUNT(*),2) AS AttritionRate
FROM employee GROUP BY Department, JobRole ORDER BY AttritionRate DESC;"""
cursor.execute(query6)
results = cursor.fetchall()
for dept, role, rate in results:
    print(f"{dept} - {role}: {rate}%")
# Pandas way
attrition_counts = df.groupby(['Department', 'JobRole'])['Attrition'].value_counts().unstack().fillna(0)
attrition_rate = (attrition_counts['Yes'] / attrition_counts.sum(axis=1) * 100).round(2)
print("Pandas:")
for idx, rate in attrition_rate.sort_values(ascending=False).items():
    print(f"{idx[0]} - {idx[1]}: {rate}%")

## 7. Correlation between monthly income and attrition

In [ ]:
print("7. Correlation between monthly income and attrition")
# SQL way
query7 = """SELECT Attrition, AVG(MonthlyIncome)
FROM employee GROUP BY Attrition;"""
cursor.execute(query7)
results = cursor.fetchall()
for attrition, avg_income in results:
    print(f"{attrition}: ${avg_income:.2f}")
# Pandas way
income_by_attrition = df.groupby('Attrition')['MonthlyIncome'].mean()
print("Pandas:")
for attrition, avg_income in income_by_attrition.items():
    print(f"{attrition}: ${avg_income:.2f}")

## 8. Performance rating vs attrition

In [ ]:
print("8. Performance rating vs attrition")
# SQL way
query8 = """
SELECT PerformanceRating, Attrition, COUNT(*)
FROM employee
GROUP BY PerformanceRating, Attrition
ORDER BY PerformanceRating DESC;
"""
cursor.execute(query8)
results = cursor.fetchall()
for rating, attrition, count in results:
    print(f"PerformanceRating: {rating}, Attrition: {attrition}, Count: {count}")
# Pandas way
perf_attrition = df.groupby(['PerformanceRating', 'Attrition']).size().unstack(fill_value=0)
print("Pandas:")
print(perf_attrition)

## 9. Job Role and Overtime vs Attrition

In [ ]:
print("9. How do Job Role and Overtime status affect employee attrition?")
# SQL version
query10 = """
SELECT JobRole, OverTime,
       ROUND(SUM(CASE WHEN Attrition='Yes' THEN 1 ELSE 0 END)*100.0/COUNT(*),2) AS AttritionRate,
       COUNT(*) AS EmployeeCount
FROM employee
GROUP BY JobRole, OverTime
ORDER BY AttritionRate DESC, EmployeeCount DESC;
"""
cursor.execute(query10)
results = cursor.fetchall()

# Convert SQL results to DataFrame
sql_df = pd.DataFrame(results, columns=['JobRole', 'OverTime', 'AttritionRate', 'EmployeeCount'])
print("Attrition by Job Role and Overtime (from SQL):")
display(sql_df)

# Pandas calculation
job_ot_attrition = (
    df.groupby(['JobRole', 'OverTime'])['Attrition']
      .value_counts()
      .unstack(fill_value=0)
)

# Add EmployeeCount and AttritionRate
job_ot_attrition['EmployeeCount'] = job_ot_attrition.sum(axis=1)
job_ot_attrition['AttritionRate'] = (job_ot_attrition['Yes'] / job_ot_attrition['EmployeeCount'] * 100).round(2)

# Reset index for readability
job_ot_attrition = job_ot_attrition.reset_index()

print("\nAttrition by Job Role and Overtime (from Pandas):")
display(job_ot_attrition)